# Boosting SLP

## Import Problem

In [2]:
import pycutest as pc
import numpy as np
import os
from collections import OrderedDict
import autograd.numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from autograd import grad
from collections import Counter, defaultdict
from itertools import compress
from scipy.io import loadmat
from scipy.optimize import linear_sum_assignment
from scipy.sparse.linalg import svds
from scipy.special import huber
from sklearn.preprocessing import StandardScaler
from time import process_time

import warnings
warnings.simplefilter('ignore')
import logging
import pickle
import time

# from cuter_util import Cuter
# from linear_solver import linearSolveTrustRegion
# from param import DustParam
# from debug_utils import pause

/home/mcq/Documents/Github/venv/lib/python3.10/site-packages/pycutest/__init__.py:28: RuntimeWarning: the PYCUTEST_CACHE environment variable is not set; current folder will be used for caching.
  warnings.warn("the PYCUTEST_CACHE environment variable is not set; current folder will be used for caching.", RuntimeWarning)


In [5]:
pc.print_available_sif_params('HS20')
p = pc.import_problem('HS20')
p_properties = pc.problem_properties('HS20')
print(p)
print(p_properties)
x = p.x0
f, grad_f = p.obj(x, gradient = True)
c, grad_c = p.cons(x, gradient = True)





p.report()

Parameters available for problem HS20:
End of parameters for problem HS20
CUTEst problem HS20 (default params) with 2 variables and 3 constraints
{'objective': 'sum of squares', 'constraints': 'quadratic', 'regular': True, 'degree': 2, 'origin': 'academic', 'internal': False, 'n': 2, 'm': 3}


{'f': 1,
 'g': 1,
 'H': 0,
 'Hprod': 0,
 'c': 1,
 'cg': 1,
 'cH': 0,
 'tsetup': 0.0,
 'trun': 0.0}

In [13]:
__PMAX = 1e3
__UP = 1e20
__LOW = -1e20
status = p.init_stats
funit = 42
fname = p.name
iout = 6
io_buffer = 11
n = p.n
m = p.m
x = p.x0
bl = p.bl
bu = p.bu
v = p.v0
cl = p.cl
cu = p.cu
equatn = p.is_eq_cons
linear = p.is_linear_cons
e_order = 1
l_order = 1
v_order = 2
iequatn = np.logical_not(equatn)
inequality_upper = np.logical_and((cu != __UP).flatten(), iequatn)
inequality_lower = np.logical_and((cl != __LOW).flatten(), iequatn)
is_lower_bound_only_constr = np.logical_and(inequality_lower, np.logical_not(inequality_upper))
is_double_bound_constr = np.logical_and(inequality_lower, inequality_upper)
bl_flag = (bl != __LOW).flatten()
bu_flag = (bu != __UP).flatten()
num_added_ineq_constr = np.sum(is_double_bound_constr) + np.sum(bl_flag) + np.sum(bu_flag)
added_iequatn = np.array([False] * num_added_ineq_constr, dtype=bool)
adjusted_equatn = np.hstack([equatn, added_iequatn])
setup_args_dict = {'status': status,'funit': funit,
                    'iout': iout,'n': n,
                    'm': m,'x': x,
                    'bl': bl,'bu': bu,
                    'v': v,'cl': cl,
                    'cu': cu,'equatn': equatn,
                    'linear': linear,
                    'e_order': e_order,
                    'l_order': l_order,
                    'v_order': v_order,
                    'bl_flag': bl_flag,
                    'bu_flag': bu_flag,
                    'adjusted_equatn': adjusted_equatn,
                    'iequatn': iequatn,
                    'inequality_lower': inequality_lower,
                    'inequality_upper': inequality_upper,
                    'is_lower_bound_only_constr': is_lower_bound_only_constr,
                    'is_double_bound_constr': is_double_bound_constr}


In [ ]:
def nlp_test(sif_dir_root, problem_name, dust_param, log_dir, result_dir):
    """
    Test one cuter problem stored under `sif_dir_root`/`problem_name`
    :param sif_dir_root: sif directory
    :param problem_name: problem name, e.g. HS101
    :param dust_param: dust param instance
    :param log_dir: output log directory
    :param result_dir: result directory to store the output
    :return: dust output
    """
    #logger.info('+' * 200)
    with Cuter(os.path.join(sif_dir_root, problem_name)) as cuter:
        try:
            print_param_dict(dust_param.dump2Dict(), None)
            start_time = time.time()
            dust_output = linearSolveTrustRegion(cuter, dust_param, None)
            execution_time = time.time() - start_time
            dust_output['problem_name'] = problem_name
            dust_output['execution_time'] = execution_time
            output_print_dict = OrderedDict(
                [('Summary for problem', dust_output['problem_name']),
                 ('Status', dust_output['status']), ('Iteration Number', dust_output['iter_num']),
                 ('Final objective', dust_output['obj_f']), ('KKT error', dust_output['kkt_error']),
                 ('Constraint violation', dust_output['constraint_violation']), 
                 ('Execute Time (s)', execution_time),
                 ('Pivot#', dust_output['pivot_cnt'])])
            print_param_dict(output_print_dict, None)
            save_output(result_dir, dust_output)
            if dust_output['status'] == 1:
                success_list.append(problem_name)
            else:
                fail_list.append(problem_name)
        except Exception as e:
            print(e)

In [ ]:
def b_nlp_test(sif_dir_root, problem_name, dust_param, log_dir, result_dir):
    